In [39]:
%matplotlib inline
%load_ext fortranmagic

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

mpl.rc('figure', figsize=(12, 7))

ran_the_first_cell = True

The fortranmagic extension is already loaded. To reload it, use:
  %reload_ext fortranmagic


<center>
  <h1>Foundations of Numerical Computing</h1>
  <h3>Scott Sanderson (GitHub: [@ssanderson](https://github.com/ssanderson) Twitter: [@scottbsanderson](https://twitter.com/scottbsanderson) )</h3>
  <h3>[https://github.com/ssanderson/foundations-of-numerical-computing](https://github.com/ssanderson/foundations-of-numerical-computing)</h3>
</center>

# About Me:

- Senior Engineer at [Quantopian](www.quantopian.com)
  - I build tools for strangers on the internet to write quantitative trading strategies.
  - Most of those tools are written using the PyData stack.
- Background in Mathematics and Philosophy
- **Twitter:** [@scottbsanderson](https://twitter.com/scottbsanderson)
- **GitHub:** [ssanderson](github.com/ssanderson)

## Introduction

The goal of this tutorial is help you effectively solve numerically-intensive problems in Python.

In practice, doing numerical programming in Python means using [`numpy`](http://www.numpy.org/) (or a library built on top of `numpy`).

Using `numpy` effectively requires changing the way that you think about and solve problems.  TODO: Sentence here?

## Educational Philosophy

TODO: Fill this in.

Programming problems require two kinds of knowledge:

- **Foundational Knowledge**
- **Domain Knowledge**

- Foundational Knowledge can be applicable across a wide variety of domains
- Domain Knowledge tends to be more limited in applicability (but it's crucial when it applies!)

# Data Structures

> Rule 5. Data dominates. If you've chosen the right data structures and organized things well, the algorithms
will almost always be self-evident. Data structures, not algorithms, are central to programming.

- *Notes on Programming in C*, by Rob Pike.

# Review: Python Lists

In [40]:
assert ran_the_first_cell, "You need to run the first cell!"

In [41]:
l = [1, 'two', 3.0, 4, 5.0, "six"]
l

[1, 'two', 3.0, 4, 5.0, 'six']

In [42]:
# Lists can be indexed like C-style arrays.
first = l[0]
second = l[1]
print("first:", first)
print("second:", second)

first: 1
second: two


In [43]:
# Negative indexing gives elements relative to the end of the list.
last = l[-1]
penultimate = l[-2]
print("last:", last)
print("second to last:", penultimate)

last: six
second to last: 5.0


In [44]:
# Lists can also be sliced, which makes a copy of elements between 
# start (inclusive) and stop (exclusive)
sublist = l[1:3]
sublist

['two', 3.0]

In [45]:
# l[:N] is equivalent to l[0:N].
first_three = l[:3]
first_three

[1, 'two', 3.0]

In [46]:
# l[3:] is equivalent to l[3:len(l)].
after_three = l[3:]
after_three

[4, 5.0, 'six']

In [47]:
# There's also a third parameter, "step", which gets every Nth element.
l = ['a', 'b', 'c', 'd', 'e', 'f', 'g','h']
l[1:7:2]

['b', 'd', 'f']

In [49]:
# Indexing with a step of -1 is a cute way to reverse a list.
l[::-1]

['h', 'g', 'f', 'e', 'd', 'c', 'b', 'a']

In [11]:
# Lists can be grown efficiently (in O(1) amortized time).
l = [1, 2, 3, 4, 5]
print("Before:", l)
l.append('six')
print("After:", l)

Before: [1, 2, 3, 4, 5]
After: [1, 2, 3, 4, 5, 'six']


In [12]:
# Comprehensions let us perform elementwise computations.
l = [1, 2, 3, 4, 5]
[x * 2 for x in l]

[2, 4, 6, 8, 10]

## Review of Review: Python Lists

- Zero-indexed sequence of values.
- Can hold values of different types in a single list.
- Slicing syntax: `l[start:stop:step]` copies elements at regular intervals from `start` to `stop`.
- Efficient (`O(1)`) appends and removes from end.
- Comprehension syntax: `[f(x) for x in l if cond(x)]`.

<center><img src="images/pacino.gif" alt="Drawing" style="width: 100%;"/></center>

## Numerical Programming in Pure Python

In [7]:
# Suppose we have some matrices...
a = [[1, 2, 3],
     [2, 3, 4],
     [5, 6, 7],
     [1, 1, 1]]

b = [[1, 2, 3, 4],
     [2, 3, 4, 5]]

In [50]:
def matmul(A, B):
    """Multiply matrix A by matrix B."""
    
    rows_out = len(A)
    cols_out = len(B[0])
    out = [[0 for col in range(cols_out)] for row in range(rows_out)]
    
    for i in range(rows_out):
        for j in range(cols_out):
            for k in range(len(B)):
                out[i][j] += A[i][k] * B[k][j]
    return out

matmul(a, b)

[[5, 8, 11, 14], [8, 13, 18, 23], [17, 28, 39, 50], [3, 5, 7, 9]]

<center><img src="images/gross.gif" alt="Drawing" style="width: 50%;"/></center>


In [23]:
%%time
matmul(a, b)

CPU times: user 20 µs, sys: 1 µs, total: 21 µs
Wall time: 23.4 µs


[[5, 8, 11, 14], [8, 13, 18, 23], [17, 28, 39, 50], [3, 5, 7, 9]]

In [53]:
import random
def random_matrix(m, n):
    out = []
    for row in range(m):
        out.append([random.random() for _ in range(n)])
    return out

randm = random_matrix(2, 3)
randm

[[0.3069394355442745, 0.24161151820144178, 0.6652130044201517],
 [0.43958386353564427, 0.5021999929762764, 0.9361093295784033]]

In [25]:
%%time
randa = random_matrix(500, 100)
randb = random_matrix(100, 500)
x = matmul(randa, randb)

CPU times: user 4.08 s, sys: 2.8 ms, total: 4.09 s
Wall time: 4.08 s


In [26]:
# Maybe that's as good as we can do?  Let's try a simpler case.
def python_dot_product(xs, ys):
    return sum(x * y for x, y in zip(xs, ys))

In [54]:
%%fortran
subroutine fortran_dot_product(xs, ys, result)
    double precision, intent(in) :: xs(:)
    double precision, intent(in) :: ys(:)
    double precision, intent(out) :: result
    
    result = sum(xs * ys)
end

In [55]:
list_data = [float(i) for i in range(100000)]
array_data = np.array(list_data)

In [56]:
%%time
python_dot_product(list_data, list_data)

CPU times: user 7.87 ms, sys: 0 ns, total: 7.87 ms
Wall time: 7.81 ms


333328333350000.0

In [57]:
%%time
fortran_dot_product(array_data, array_data)

CPU times: user 133 µs, sys: 5 µs, total: 138 µs
Wall time: 141 µs


333328333350000.0

<center><img src="images/sloth.gif" alt="Drawing" style="width: 1080px;"/></center>

## Why is the Python Version so Much Slower?

In [62]:
# Dynamic typing.
def multiply_elementwise(xs, ys):
    return [x * y for x, y in zip(xs, ys)]

multiply_elementwise([1, 2, 3, 4], [1, 2 + 0j, 3.0, 'four'])
#[type(x) for x in _]

[int, complex, float, str]

In [59]:
# Interpretation overhead.
source_code = 'a + b * c'
bytecode = compile(source_code, '', 'eval')
import dis; dis.dis(bytecode)

  1           0 LOAD_NAME                0 (a)
              3 LOAD_NAME                1 (b)
              6 LOAD_NAME                2 (c)
              9 BINARY_MULTIPLY
             10 BINARY_ADD
             11 RETURN_VALUE


## Review: Why is the Python Version so Slow?
- Dynamic typing means that every single operation requires dispatching on the input type.
- Having an interpreter means that every instruction is fetched and dispatched at runtime.
- Other overheads:
  - Arbitrary-size integers.
  - Reference-counted garbage collection.

> This is the paradox that we have to work with when we're doing scientific or numerically-intensive Python. What makes Python fast for development -- this high-level, interpreted, and dynamically-typed aspect of the language -- is exactly what makes it slow for code execution.

- Jake VanderPlas, [*Losing Your Loops: Fast Numerical Computing with NumPy*](https://www.youtube.com/watch?v=EEUXKG97YRw)

# What Do We Do?

<center><img src="images/runaway.gif" alt="Drawing" style="width: 50%;"/></center>

<center><img src="images/thisisfine.gif" alt="Drawing" style="width: 1080px;"/></center>

- Normal Python is slow for numerical computation because it performs dynamic dispatch on every operation we perform...

- ...but often, we just want to do the same thing over and over in a loop!

- If we don't need Python's dynamicism, can we find a way to not pay (much) for it?

- **Idea:** Dispatch **once per logical operation** instead of **once per element**.

# Numpy

In [112]:
import numpy as np

data = np.array([1, 2, 3, 4, 5, 6, 7, 8])
data

array([1, 2, 3, 4, 5, 6, 7, 8])

In [113]:
# Numpy provides operators that "vectorize" over the entire array.
data + data

array([ 2,  4,  6,  8, 10, 12, 14, 16])

In [114]:
%%time
# Naive dot product
(array_data * array_data).sum()

CPU times: user 569 µs, sys: 28 µs, total: 597 µs
Wall time: 316 µs


333328333350000.0

In [115]:
%%time
# Built-in dot product.
array_data.dot(array_data)

CPU times: user 240 µs, sys: 12 µs, total: 252 µs
Wall time: 149 µs


333328333350000.0

In [116]:
%%time
fortran_dot_product(array_data, array_data)

CPU times: user 105 µs, sys: 5 µs, total: 110 µs
Wall time: 113 µs


333328333350000.0

In [117]:
# Numpy won't allow us to write a string into an int array.
data[0] = "foo"

ValueError: invalid literal for int() with base 10: 'foo'

In [118]:
# We also can't grow an array once it's created.
data.append(3)

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [123]:
# We **can** reshape an array as long as the number of elements doesn't change.
two_by_two = data.reshape(4, 2)
two_by_two

array([[1, 2],
       [3, 4],
       [5, 6],
       [7, 8]])

Numpy arrays are:

- Fixed-type

- Size-immutable

- Multi-dimensional

- Fast\*

\* If you use them correctly.

# Exercises - Finding Help and Measuring Performance

## Solving Problems with Numpy - Overview

Numpy can be fast because it allows us to specify operations to perform against an entire array.

We can break down the tools provided by numpy into a few categories:

1. Universal Functions (UFuncs)
2. Selection
3. Reductions
4. Broadcasting

Taken together, these operations form an expressive toolbox that can solve a diverse set of problems.

## Creating Arrays

Numpy provides many functions for creating arrays.

We'll be using the following functions throughout the tutorial:

- `np.array`: Construct an array from a list of Python objects.
- `np.arange`: Numpy equivalent of the `range` function.
- `np.linspace`: Create an array from evenly-spaced points between two values.
- `np.geomspace`: Create an array from logarithmically-spaced points between two values.
- `np.full`/`np.ones`/`np.zeros`: Create an array of a given shape with a constant value.
- `np.full_like`/`np.ones_like`/`np.zeros_like`: Same, but using the shape of another array.

In [135]:
np.(1, 10, 30)

array([ 1.        ,  1.08263673,  1.1721023 ,  1.268961  ,  1.3738238 ,
        1.48735211,  1.61026203,  1.74332882,  1.88739182,  2.04335972,
        2.21221629,  2.39502662,  2.5929438 ,  2.8072162 ,  3.03919538,
        3.29034456,  3.56224789,  3.85662042,  4.17531894,  4.52035366,
        4.89390092,  5.29831691,  5.73615251,  6.21016942,  6.72335754,
        7.27895384,  7.88046282,  8.53167852,  9.23670857, 10.        ])

## Creating Arrays (cont'd)

- `np.eye`/`np.identity`/`np.diag`: Arrays with values on the diagonal. Useful for linear algebra.
- `np.random`: Randomly-generated values from various distributions.
- `np.load`/`np.save`: Load and save values from disk.
- `pandas.read_csv`: Read values from a CSV. Returns a [`pandas.DataFrame`](https://pandas.pydata.org/).

## Exercise  - Creating Arrays

## Solving Problems with Numpy - Universal Functions

Universal functions (ufuncs) are functions that can be applied across one or more arrays of the same shape.

They come in two main varieties:

- **Unary UFuncs** apply a 1-argument function to every element of an array.
- **Binary UFuncs** apply a 2-argument function to corresponding elements from two arrays.

In [127]:
np.logspace?

In [96]:
arr = np.array([1, 2, 3, 4, 5, 6])
arr.

In [36]:
data = np.arange(15).reshape(3, 5)
data

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [37]:
# Binary operators.
data * data

array([[  0,   1,   4,   9,  16],
       [ 25,  36,  49,  64,  81],
       [100, 121, 144, 169, 196]])

In [38]:
# Unary functions.
np.sqrt(data)

array([[ 0.        ,  1.        ,  1.41421356,  1.73205081,  2.        ],
       [ 2.23606798,  2.44948974,  2.64575131,  2.82842712,  3.        ],
       [ 3.16227766,  3.31662479,  3.46410162,  3.60555128,  3.74165739]])

In [39]:
# Comparison operations
(data % 3) == 0

array([[ True, False, False,  True, False],
       [False,  True, False, False,  True],
       [False, False,  True, False, False]], dtype=bool)

In [40]:
# Boolean combinators.
((data % 2) == 0) & ((data % 3) == 0)

array([[ True, False, False, False, False],
       [False,  True, False, False, False],
       [False, False,  True, False, False]], dtype=bool)

In [41]:
# as of python 3.5, @ is matrix-multiply
data @ data.T

array([[ 30,  80, 130],
       [ 80, 255, 430],
       [130, 430, 730]])

# UFuncs Review

- UFuncs provide efficient elementwise operations applied across one or more arrays.
- Arithmetic Operators (`+`, `*`, `/`)
- Comparisons (`==`, `>`, `!=`)
- Boolean Operators (`&`, `|`, `^`)
- Trigonometric Functions (`sin`, `cos`)
- Transcendental Functions (`exp`, `log`)

# Selections

We often want to perform an operation on just a subset of our data.

In [42]:
sines = np.sin(np.linspace(0, 3.14, 10))
cosines = np.cos(np.linspace(0, 3.14, 10))
sines

array([ 0.        ,  0.34185385,  0.64251645,  0.86575984,  0.98468459,
        0.98496101,  0.8665558 ,  0.64373604,  0.34335012,  0.00159265])

In [43]:
# Slicing works with the same semantics as Python lists.
sines[0]

0.0

In [44]:
sines[:3]  # First three elements  

array([ 0.        ,  0.34185385,  0.64251645])

In [45]:
sines[5:]  # Elements from 5 on.

array([ 0.98496101,  0.8665558 ,  0.64373604,  0.34335012,  0.00159265])

In [46]:
sines[::2]  # Every other element.

array([ 0.        ,  0.64251645,  0.98468459,  0.8665558 ,  0.34335012])

In [47]:
# More interesting: we can index with boolean arrays to filter by a predicate.
print("sines:\n", sines)
print("sines > 0.5:\n", sines > 0.5)
print("sines[sines > 0.5]:\n", sines[sines > 0.5])

sines:
 [ 0.          0.34185385  0.64251645  0.86575984  0.98468459  0.98496101
  0.8665558   0.64373604  0.34335012  0.00159265]
sines > 0.5:
 [False False  True  True  True  True  True  True False False]
sines[sines > 0.5]:
 [ 0.64251645  0.86575984  0.98468459  0.98496101  0.8665558   0.64373604]


In [48]:
# We index with lists/arrays of integers to select values at those indices.
print(sines)
sines[[0, 4, 7]]

[ 0.          0.34185385  0.64251645  0.86575984  0.98468459  0.98496101
  0.8665558   0.64373604  0.34335012  0.00159265]


array([ 0.        ,  0.98468459,  0.64373604])

In [49]:
# Index arrays are often used for sorting one or more arrays.
unsorted_data = np.array([1, 3, 2, 12, -1, 5, 2])

In [50]:
sort_indices = np.argsort(unsorted_data)
sort_indices

array([4, 0, 2, 6, 1, 5, 3])

In [51]:
unsorted_data[sort_indices]

array([-1,  1,  2,  2,  3,  5, 12])

In [52]:
market_caps = np.array([12, 6, 10, 5, 6])  # Presumably in dollars?
assets = np.array(['A', 'B', 'C', 'D', 'E'])

In [53]:
# Sort assets by market cap by using the permutation that would sort market caps on ``assets``.
sort_by_mcap = np.argsort(market_caps)
assets[sort_by_mcap]

array(['D', 'B', 'E', 'C', 'A'],
      dtype='<U1')

In [54]:
# Indexers are also useful for aligning data.
print("Dates:\n", repr(event_dates))
print("Values:\n", repr(event_values))
print("Calendar:\n", repr(calendar))

Dates:
 array(['2017-01-06', '2017-01-07', '2017-01-08'], dtype='datetime64[D]')
Values:
 array([10, 15, 20])
Calendar:
 array(['2017-01-03', '2017-01-04', '2017-01-05', '2017-01-06',
       '2017-01-09', '2017-01-10', '2017-01-11', '2017-01-12',
       '2017-01-13', '2017-01-17', '2017-01-18', '2017-01-19',
       '2017-01-20', '2017-01-23', '2017-01-24', '2017-01-25',
       '2017-01-26', '2017-01-27', '2017-01-30', '2017-01-31', '2017-02-01'], dtype='datetime64[D]')


In [55]:
print("Raw Dates:", event_dates)
print("Indices:", calendar.searchsorted(event_dates))
print("Forward-Filled Dates:", calendar[calendar.searchsorted(event_dates)])

Raw Dates: ['2017-01-06' '2017-01-07' '2017-01-08']
Indices: [3 4 4]
Forward-Filled Dates: ['2017-01-06' '2017-01-09' '2017-01-09']


On multi-dimensional arrays, we can slice along each axis independently.

In [56]:
data = np.arange(25).reshape(5, 5)
data

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [57]:
data[:2, :2]  # First two rows and first two columns.

array([[0, 1],
       [5, 6]])

In [58]:
data[:2, [0, -1]]  # First two rows, first and last columns.

array([[0, 4],
       [5, 9]])

In [59]:
data[(data[:, 0] % 2) == 0]  # Rows where the first column is divisible by two.

array([[ 0,  1,  2,  3,  4],
       [10, 11, 12, 13, 14],
       [20, 21, 22, 23, 24]])

# Selections Review

- Indexing with an integer removes a dimension.
- Slicing operations work on Numpy arrays the same way they do on lists.
- Indexing with a boolean array filters to True locations.
- Indexing with an integer array selects indices along an axis.
- Multidimensional arrays can apply selections independently along different axes.

## Reductions

Functions that reduce an array to a scalar.

$Var(X) = \frac{1}{N}\sqrt{\sum_{i=1}^N (x_i - \bar{x})^2}$

In [60]:
def variance(x):
    return ((x - x.mean()) ** 2).sum() / len(x)

In [61]:
variance(np.random.standard_normal(1000))

1.0638195544963331

- `sum()` and `mean()` are both **reductions**.

- In the simplest case, we use these to reduce an entire array into a single value...

In [62]:
data = np.arange(30)
data.mean()

14.5

- ...but we can do more interesting things with multi-dimensional arrays.

In [63]:
data = np.arange(30).reshape(3, 10)
data

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

In [64]:
data.mean()

14.5

In [65]:
data.mean(axis=0)

array([ 10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.])

In [66]:
data.mean(axis=1)

array([  4.5,  14.5,  24.5])

## Reductions Review

- Reductions allow us to perform efficient aggregations over arrays.
- We can do aggregations over a single axis to collapse a single dimension.
- Many built-in reductions (`mean`, `sum`, `min`, `max`, `median`, ...).

# Broadcasting

In [67]:
row = np.array([1, 2, 3, 4])
column = np.array([[1], [2], [3]])
print("Row:\n", row, sep='')
print("Column:\n", column, sep='')

Row:
[1 2 3 4]
Column:
[[1]
 [2]
 [3]]


In [68]:
row + column

array([[2, 3, 4, 5],
       [3, 4, 5, 6],
       [4, 5, 6, 7]])

<center><img src="images/broadcasting.png" alt="Drawing" style="width: 60%;"/></center>

<h5>Source: http://www.scipy-lectures.org/_images/numpy_broadcasting.png</h5>

In [69]:
# Broadcasting is particularly useful in conjunction with reductions.
print("Data:\n", data, sep='')
print("Mean:\n", data.mean(axis=0), sep='')
print("Data - Mean:\n", data - data.mean(axis=0), sep='')

Data:
[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]]
Mean:
[ 10.  11.  12.  13.  14.  15.  16.  17.  18.  19.]
Data - Mean:
[[-10. -10. -10. -10. -10. -10. -10. -10. -10. -10.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 10.  10.  10.  10.  10.  10.  10.  10.  10.  10.]]


# Broadcasting Review

- Numpy operations can work on arrays of different dimensions as long as the arrays' shapes are still "compatible".
- Broadcasting works by "tiling" the smaller array along the missing dimension.
- The result of a broadcasted operation is always at least as large in each dimension as the largest array in that dimension.

# Numpy Review

- Numerical algorithms are slow in pure Python because the overhead of interpretation and dynamic dispatch dominates our runtime.

- Numpy solves this problem by:
  1. Imposing additional restrictions on the contents of arrays.
  2. Moving the inner loops of our algorithms into compiled C code.

- Using Numpy effectively often requires reworking an algorithms to use vectorized operations instead of for-loops, but the resulting operations are usually simpler, clearer, and faster than the pure Python equivalent.

In [95]:
arr = np.array([1, 2, 3, 4, 5, 6], dtype='int32').reshape(2, 3)
print("Array:\n", arr, sep='')
print("===========")
print("DType:", arr.dtype)
print("Shape:", arr.shape)
print("Strides:", arr.strides)
print("Data:", arr.data.tobytes())

Array:
[[1 2 3]
 [4 5 6]]
DType: int32
Shape: (2, 3)
Strides: (12, 4)
Data: b'\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x04\x00\x00\x00\x05\x00\x00\x00\x06\x00\x00\x00'
